## Guider mode image compare
This is intended to unpack the rawStamps from guider mode FITS files and \
compare the unpacked stamp to the IMAGE mode stamp. \
Craig Lage - 16-Apr-25

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from lsst.summit.utils.plotting import plot
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath
import matplotlib.colors as colors
from lsst.summit.utils import getQuantiles
import scipy.ndimage as ndi

# Get the main header and the information it contains

In [ ]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    roiCol = hdr0['ROICOL']
    roiRow = hdr0['ROIROW']
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDRC']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor]

# Now define the code to unpack the rawStamps:

In [ ]:
def unpackStamps(hduNum):
    data = np.array(hdu_list[hduNum].data[0]).astype('>u4')[0]
    data.byteswap(inplace=True)
    totalCols = roiCols + roiUnder
    size = roiRows * totalCols
    out = np.zeros([16, size], dtype=int)
    image_out = np.zeros([16, roiRows, roiCols], dtype=int)
    for n in range(size):
        # Get 9 32 bit words of data
        res = ''
        for i in range(9):
            d = data[(size - n) * 9 - i - 1]
            d = format(d, '#034b')
            d = d.split('b')[1]
            res += d
        # Now extract 16 18 bit words from the data
        for i in range(16):
            bin_value = res[i * 18:(i + 1) * 18]
            int_value = int(bin_value, 2)
            final_value = int_value ^ xor
            out[i,n] = final_value  
    for i in range(16):
        reshaped = out[i,:].reshape(roiRows, totalCols)
        image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:roiCols]))
    return image_out

# Build a movie of a single stamp

In [ ]:

dayObs = 20250414
seqNum = 660
seg = 13
expId = int(f"{dayObs}{seqNum:05d}")
raft = 'R00'
ccd = 'SG0'
"""
dayObs = 20250415
seqNum = 140
seg = 7
expId = int(f"{dayObs}{seqNum:05d}")
raft = 'R00'
ccd = 'SG0'


dayObs = 20250415
seqNum = 140
seg = 0
expId = int(f"{dayObs}{seqNum:05d}")
raft = 'R00'
ccd = 'SG1'
"""


n = 1
filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"

rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
[roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
# First get the packed data:
hduNum = 2 * n + 1
hdrn = hdu_list[hduNum].header
image_out = unpackStamps(hduNum)
unpacked_arr = image_out[seg]
# Then get the image data:
hduNum = 2 * n + 2
image_arr = hdu_list[hduNum].data

In [ ]:
fig, axs = plt.subplots(1,3)
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"Unpacking comparison {expId} {raft} {ccd} {seg}", fontsize=12, y=0.8)
quantiles = getQuantiles(unpacked_arr, 256)
norm = colors.BoundaryNorm(quantiles, 256)
axs[0].set_title("Unpacked segment")
axs[0].imshow(unpacked_arr, interpolation='nearest', 
                                  origin='lower', cmap='Greys', aspect='equal', norm=norm)
quantiles = getQuantiles(image_arr, 256)
norm = colors.BoundaryNorm(quantiles, 256)
axs[1].set_title("Image segment")
axs[1].imshow(image_arr, interpolation='nearest', 
                                  origin='lower', cmap='Greys', aspect='equal', norm=norm)

diff = unpacked_arr - image_arr
quantiles = getQuantiles(diff, 256)
norm = colors.BoundaryNorm(quantiles, 256)
axs[2].set_title(f"Diff: Max = {np.max(diff)}")
axs[2].imshow(diff, interpolation='nearest', 
                                  origin='lower', cmap='Greys', aspect='equal', norm=norm)
plt.savefig(f"/home/c/cslage/u/Guider_Mode/Unpacking Check_{expId}_{raft}_{ccd}_{seg}.png")

In [ ]:
plt.plot(unpacked_arr[0,:], color='red')
plt.plot(image_arr[0,:], color='blue', ls='--')

In [ ]:
print(f"Image_out median = {np.median(image_arr)}")
for seg in range(16):
    print(f" For Seg={seg}, Unpacked median = {np.median(image_out[seg])}")